## DEPENDENCIES INSTALL

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!nvidia-smi

Sun Sep  3 14:05:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## LLaMA2-7B-Chat


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config insta

In [ ]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
!nvidia-smi

Sun Sep  3 14:07:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    27W /  70W |   8097MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


## **PROMPT FORMAT FOR LLAMA-2**
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

<</SYS>>

{prompt} [/INST]


In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
instruction = "How to make ISRO is successful in making Satellites and Lunar Missions?"

get_prompt(instruction)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nHow to make ISRO is successful in making Satellites and Lunar Missions?[/INST]"

In [ ]:
instruction = "Summarize the following text : {text}"

system_prompt = "You are an expert and summarization and reducing the number of words used"

get_prompt(instruction, system_prompt)

'[INST]<<SYS>>\nYou are an expert and summarization and reducing the number of words used\n<</SYS>>\n\nSummarize the following text : {text}[/INST]'

## LangChain

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


In [ ]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

## Translation

In [ ]:
system_prompt = "You are an advanced assistant and transalation system "
instruction = "Convert the following text from English to Spanish:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an advanced assistant and transalation system 
<</SYS>>

Convert the following text from English to Spanish:

 {text}[/INST]


In [ ]:
text = "How to make chatbot in easiest way possible?"
output = llm_chain.run(text)

parse_text(output)

  ¡Claro! Here's the translation of "How to make chatbot in easiest way possible?" from English to
Spanish:  ¿Cómo crear chatbot de la manera más fácil posible?




### Summarization

In [ ]:
instruction = "Summarize the following article : {text}"
system_prompt = "You are an expert in summarization of articles and expressing the important information"

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an expert in summarization of articles and expressing the important information
<</SYS>>

Summarize the following article : {text}[/INST]


In [ ]:
def count_words(input_string):
    words = input_string.split(" ")
    return len(words)

text = '''
Chandrayaan-3 consists of an indigenous Lander module (LM), Propulsion module (PM) and a Rover with an objective of developing and demonstrating new technologies required for Inter planetary missions. The Lander will have the capability to soft land at a specified lunar site and deploy the Rover which will carry out in-situ chemical analysis of the lunar surface during the course of its mobility. The Lander and the Rover have scientific payloads to carry out experiments on the lunar surface. The main function of PM is to carry the LM from launch vehicle injection till final lunar 100 km circular polar orbit and separate the LM from PM. Apart from this, the Propulsion Module also has one scientific payload as a value addition which will be operated post separation of Lander Module. The launcher identified for Chandrayaan-3 is LVM3 M4 which will place the integrated module in an Elliptic Parking Orbit (EPO) of size ~170 x 36500 km.

The mission objectives of Chandrayaan-3 are:

To demonstrate Safe and Soft Landing on Lunar Surface
To demonstrate Rover roving on the moon and
To conduct in-situ scientific experiments.
To achieve the mission objectives, several advanced technologies are present in Lander such as,

Altimeters: Laser & RF based Altimeters
Velocimeters: Laser Doppler Velocimeter & Lander Horizontal Velocity Camera
Inertial Measurement: Laser Gyro based Inertial referencing and Accelerometer package
Propulsion System: 800N Throttleable Liquid Engines, 58N attitude thrusters & Throttleable Engine Control Electronics
Navigation, Guidance & Control (NGC): Powered Descent Trajectory design and associate software elements
Hazard Detection and Avoidance: Lander Hazard Detection & Avoidance Camera and Processing Algorithm
Landing Leg Mechanism.
To demonstrate the above said advanced technologies in earth condition, several Lander special tests have been planned and carried out successfully viz.

Integrated Cold Test - For the demonstration of Integrated Sensors & Navigation performance test using helicopter as test platform
Integrated Hot test – For the demonstration of closed loop performance test with sensors, actuators and NGC using Tower crane as test platform
Lander Leg mechanism performance test on a lunar simulant test bed simulating different touch down conditions.'''

count_words(text)

331

In [ ]:
output = llm_chain.run(text)
print(count_words(output))
parse_text(output)

275
  Chandrayaan-3 is an Indian space mission aimed at demonstrating new technologies for
interplanetary missions. The mission consists of a lander module (LM), propulsion module (PM), and a
rover, all of which are equipped with scientific payloads to conduct experiments on the lunar
surface. The main function of the PM is to carry the LM from launch to final lunar orbit and
separate it from the PM. The PM also has a scientific payload for post-separation operation. The
launch vehicle identified for Chandrayaan-3 is the LVM3 M4, which will place the integrated module
in an elliptic parking orbit (EPO) of size ~170 x 36500 km.  The mission objectives of Chandrayaan-3
are:  1. Safe and soft landing on the lunar surface 2. Rover roving on the moon 3. Conducting in-
situ scientific experiments  To achieve these objectives, several advanced technologies are employed
in the Lander, including:  1. Altimeters (Laser & RF based) 2. Velocimeters (Laser Doppler
Velocimeter & Lander Horizontal Ve

## Simple Chatbot

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [ ]:
instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input}"
system_prompt = "You are a helpful chat assistant.You always read the chat and respond and then stop. Read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
You are a helpful chat assistant.You always read the chat and respond and then stop. Read the chat history to get context
<</SYS>>

Chat History:

{chat_history} 

User: {user_input}[/INST]


In [ ]:

prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
llm_chain.predict(user_input="Hi, I am Karndeep. How are you?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful chat assistant.You always read the chat and respond and then stop. Read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, I am Karndeep. How are you?[/INST]

> Finished chain.


"  Hello Karndeep! I'm doing well, thank you for asking! It's great to chat with you today. How can I assist you?"

In [ ]:
llm_chain.predict(user_input="How you are helpful in assisting me?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful chat assistant.You always read the chat and respond and then stop. Read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, I am Karndeep. How are you?
AI:   Hello Karndeep! I'm doing well, thank you for asking! It's great to chat with you today. How can I assist you? 

User: How you are helpful in assisting me?[/INST]

> Finished chain.


"  AI: I'd be happy to help you in any way I can! Is there something specific you need assistance with or a task you'd like me to help you complete? Please let me know and I'll do my best to assist you."

In [ ]:
llm_chain.predict(user_input="Can you help me to write step by step approach to build a successful online business?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful chat assistant.You always read the chat and respond and then stop. Read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, I am Karndeep. How are you?
AI:   Hello Karndeep! I'm doing well, thank you for asking! It's great to chat with you today. How can I assist you?
Human: How you are helpful in assisting me?
AI:   AI: I'd be happy to help you in any way I can! Is there something specific you need assistance with or a task you'd like me to help you complete? Please let me know and I'll do my best to assist you. 

User: Can you help me to write step by step approach to build a successful online business?[/INST]

> Finished chain.


"  AI: Of course, I'd be happy to help you with that! Building a successful online business requires a well-thought-out approach that covers various aspects. Here's a step-by-step approach to help you get started:\n\nStep 1: Define Your Niche\n\n* Identify your area of expertise and passion.\n* Research your target audience and their needs.\n* Choose a niche that aligns with your expertise and has a sufficient audience.\n\nStep 2: Set Clear Goals\n\n* Define your business goals and objectives.\n* Determine your target audience and their needs.\n* Set specific, measurable, achievable, relevant, and time-bound (SMART) goals.\n\nStep 3: Conduct Market Research\n\n* Analyze your competition and their strategies.\n* Identify potential customers and their buying habits.\n* Determine the demand for your products or services in the market.\n\nStep 4: Create a Business Plan\n\n* Develop a detailed business plan that outlines your goals, strategies, and tactics.\n* Define your business model and